# Novelty Algo

## Algorithm to Automate Novelty Scoring

### Import Packages

In [74]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd 
from pandas import DataFrame
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

### Put Data from Excel Sheet into Dataframes

In [67]:
# individual df's for each sheet
data_cup = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Cup")
data_key = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Key")
data_rope = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Rope")
data_brick = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Brick")
data_chair = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Chair")
data_pencil = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Pencil")
data_shoe = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Shoe")


### Preprocessing

In [78]:
def process_text(text, stem=True):
    # tokenize text, lemmanize words, removing punctuation, remove stop words, lowercase all words

    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word not in stopwords.words('english')]

    tokens = [w.lower() for w in tokens]

    if stem:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
 
    return tokens

### Functions

In [79]:
def get_id_list(df):
    id_list = df['id'].unique()
    id_list = sorted(id_list)
    return id_list

In [80]:
def get_cleaned_responses(df):
    # id_df = df[df.id == id]
    df_processed = df.copy(deep=True)
    responses = df['response'].tolist()

    # make list of processed responses
    for response in range(len(responses)):
        responses[response] = process_text(responses[response], True)

    # add list as column in df
    df_processed['response_processed'] = responses

    return df_processed

In [81]:
test = get_cleaned_responses(data_brick)
test

,id,stim,response,response_processed
0,1476,Brick,as a foundation,[foundation]
1,1476,Brick,as a weapon,[weapon]
2,1476,Brick,as a bowl,[bowl]
3,1718,Brick,build a house,"[build, house]"
4,1718,Brick,break a window,"[break, window]"
5,1718,Brick,line a fireplace,"[line, fireplace]"
6,1718,Brick,line a road,"[line, road]"
7,1691,Brick,build house,"[build, house]"
8,1691,Brick,break window,"[break, window]"
9,1691,Brick,weight for workout,"[weight, workout]"


### Novelty Algo

In [76]:
corpus = api.load('text8')
word_model = Word2Vec(corpus)

[==================================================] 100.0% 31.6/31.6MB downloaded


In [77]:
def get_novelty(df):
    novel_rating_df = get_cleaned_responses(df)

    # implement algo

    # new column with novelty rating
    return novel_rating_df

Algo Design Brainstorming:
* Word2vec, average similarity between words in two responses
* Problems:
    * phrases vs words
    * compare between just two responses or the whole list of responses
* Current Algo Idea:
    * :metal:
    * for a response
        * take each word and find the similarity between another response's word[s]
        * if more than one word in compared response, find average of the set of similarities found 
        * if more than one word in tested response, find average of the set of similarities found
        * end with list of a response and its similarity with every other response
        * average similarity values in that list, this is the novelty rating
        * the lower the value, more "novel" b/c furthee distance relative to other words seen in responses
        * add value to list of novelty rating
        * add list as column in df
* Things to Look Into:
    * doc2Vec - look into sentence vector
        * follow similar logic to above but no need to average for multiple words
    * Word2Vec + SIF + Cosine Similarity
    * Word2Vec + WMD 

* To Do List
- [x] write preprocessing methods
- [x] write out initial algo
- [ ] implement algo idea
- [ ] look into Word2Vec + SIF + Cosine Similarity
- [ ] look into Word2Vec + WMD 